This notebook extracts the location of 512x512 crops of main images and saves it in a DataFrame.
The extraction is based on the standar deviation of pixels.
..
it takes about 9 hours to run.


In [ ]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras

In [ ]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')

In [ ]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return cv2.resize(img/255.0, (512, 512))

In [ ]:
def get_patch2(os_obj, tl_pixel, patch_shape):
    return np.array(os_obj.read_region(tl_pixel, 0, patch_shape).convert("RGB"))

final_data = pd.DataFrame()
ids = []
for n in range(df.shape[0]):
    img_path = "../input/mayo-clinic-strip-ai/train/"+df.iloc[n]['image_id']+".tif"
    label = df.iloc[n]['label']
    i_id  = df.iloc[n]['image_id']
    ids.append(i_id)
    img  = openslide.open_slide(img_path)
    series = pd.Series()
    print(n)
    for i in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
        for j in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
            im = get_patch2(img , (i,j), (512,512) )
            if im.std()>40.0:
                s = pd.Series(data = [(i,j)])
                series = series.append(s,ignore_index=True)
                series = series.reset_index()
                series = series.drop("index",axis=1)
    final_data = pd.concat([final_data , series],axis=1)
    final_data.columns = ids


In [ ]:
final_data.T.to_csv('./navigation_v2.csv')